<a href="https://colab.research.google.com/github/sakurag118/ELECTIVA-TECNICA-I-C-DATOS-/blob/main/SQLite_y_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestión Básica de Inventario con SQLite3

**Autor: Yuliet Sakura Garcia Fonseca**
Bogotá, Colombia


**Instalar la Librería**

In [16]:
# %%shell le dice a Colab que vamos a usar un comando del sistema, no Python.
# 'sudo apt-get install sqlite3' instala la herramienta de bases de datos.
%%shell
sudo apt-get install sqlite3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


**Crear la Base de Datos y la Tabla**

In [17]:
## Crear la base de datos, tabla e insertar 10,000 productos

import sqlite3 # Librería para la base de datos.
import random  # Librería para generar datos al azar.
import os      # Librería para interactuar con el sistema operativo (archivos).

# 1. Eliminar el archivo de la base de datos para empezar completamente de cero
# Esto garantiza que no haya problemas de bloqueos ni IDs duplicados de ejecuciones anteriores.
if os.path.exists('productos.db'):
    os.remove('productos.db')
    print("Archivo de base de datos anterior eliminado.")

# 2. Abrir la conexión y crear el cursor
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# 3. Crear una tabla llamada 'inventario'
c.execute('''
CREATE TABLE inventario (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    categoria TEXT,
    precio DECIMAL(10, 2),
    cantidad INTEGER,
    proveedor TEXT,
    fecha_ingreso DATE,
    ubicacion TEXT
)
''')

# 4. Función para generar un producto (solo 7 datos, el ID lo genera la DB)
def generar_producto():
    nombres = ["Producto", "Artículo", "Objeto"]
    categorias = ["Electrónica", "Hogar", "Jardín", "Moda", "Deportes"]
    proveedores = ["Proveedor1", "Proveedor2", "Proveedor3"]
    ubicaciones = ["Almacén A", "Almacén B", "Almacén C"]

    nombre = random.choice(nombres) + str(random.randint(10000, 99999))
    categoria = random.choice(categorias)
    precio = round(random.uniform(10.0, 500.0), 2)
    cantidad = random.randint(1, 100)
    proveedor = random.choice(proveedores)
    fecha_ingreso = f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
    ubicacion = random.choice(ubicaciones)

    # Devolvemos una tupla de SOLO 7 DATOS (sin el ID)
    return (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)

# 5. Generar la lista de 10,000 productos
productos = [generar_producto() for i in range(10000)]

# 6. Insertar los 10,000 productos usando executemany
c.executemany('''
INSERT INTO inventario (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', productos)

# 7. Guardar cambios y cerrar la conexión
conn.commit()
conn.close()
print("Base de datos creada y poblada con éxito.")

Archivo de base de datos anterior eliminado.
Base de datos creada y poblada con éxito.


**Consultas y Uso de Pandas**

In [18]:
## Consulta para mostrar todos los productos creados
import sqlite3

# Reabrir la conexión
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# c.execute: Ejecuta el comando SQL 'SELECT * FROM inventario' (dame todos los productos de la tabla inventario)
c.execute('SELECT * FROM inventario')
# c.fetchall(): Trae todos los resultados que encontró la consulta
productos = c.fetchall()

# Un bucle for para imprimir cada producto
# Esto es lento y solo muestra texto, por eso usamos Pandas más abajo.
# for producto in productos:
#     print(producto)

# Cerrar la conexión
conn.close()

## Realizar consulta y transferir a dataframe
import sqlite3
import pandas as pd # Importamos PANDAS, la librería para crear DataFrames (tablas bonitas)

# Reabrir la conexión
conn = sqlite3.connect('productos.db')

#P2.2: Usamos pd.read_sql_query().
# Esto hace la consulta Y la convierte en una tabla (DataFrame) al mismo tiempo. ¡Genial!
query = "SELECT * FROM inventario WHERE precio > 450"
df_productos = pd.read_sql_query(query, conn)

# Mostrar el DataFrame (la tabla bonita)
print("Productos con precio > 450:")
print(df_productos)
df_productos.head() # Muestra las primeras 5 filas (rows)

# Cerrar la conexión
conn.close()

Productos con precio > 450:
        id         nombre    categoria  precio  cantidad   proveedor  \
0        6    Objeto37961         Moda  475.34        97  Proveedor3   
1       25    Objeto99484  Electrónica  468.27        82  Proveedor3   
2       30  Artículo30443       Jardín  485.21        11  Proveedor1   
3       40    Objeto66840       Jardín  476.33        75  Proveedor1   
4       45  Producto91834       Jardín  485.79         4  Proveedor2   
...    ...            ...          ...     ...       ...         ...   
1001  9950    Objeto83276       Jardín  475.62       100  Proveedor2   
1002  9951  Producto15424       Jardín  464.18         1  Proveedor3   
1003  9956  Producto36621         Moda  482.33        61  Proveedor3   
1004  9986  Artículo29989         Moda  452.70        91  Proveedor2   
1005  9999    Objeto26962       Jardín  474.48        97  Proveedor2   

     fecha_ingreso  ubicacion  
0       2023-03-13  Almacén C  
1       2023-01-03  Almacén B  
2       202

In [19]:
## Nueva consulta (Respuesta a P2.3)
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# 📝 Modificación para P2.3: Cambiar la condición 'precio > 450 AND cantidad > 25'
# a 'precio > 480 OR cantidad < 5'
query_modificada = "SELECT * FROM inventario WHERE precio > 480 OR cantidad < 5"

# Realizar la consulta y cargar los resultados en un DataFrame
df_productos_modificado = pd.read_sql_query(query_modificada, conn)

# Mostrar el DataFrame
print("Productos con precio > 480 O cantidad < 5:")
print(df_productos_modificado)

# Cerrar la conexión
conn.close()

Productos con precio > 480 O cantidad < 5:
       id         nombre    categoria  precio  cantidad   proveedor  \
0       2    Objeto21395     Deportes  147.99         2  Proveedor2   
1      30  Artículo30443       Jardín  485.21        11  Proveedor1   
2      45  Producto91834       Jardín  485.79         4  Proveedor2   
3      48    Objeto22855  Electrónica  494.52        25  Proveedor3   
4      55  Producto55025         Moda   25.59         1  Proveedor3   
..    ...            ...          ...     ...       ...         ...   
809  9951  Producto15424       Jardín  464.18         1  Proveedor3   
810  9956  Producto36621         Moda  482.33        61  Proveedor3   
811  9957    Objeto58815         Moda  161.07         4  Proveedor3   
812  9982    Objeto89411     Deportes  152.81         2  Proveedor3   
813  9985    Objeto53990        Hogar  211.49         2  Proveedor2   

    fecha_ingreso  ubicacion  
0      2023-10-13  Almacén A  
1      2023-06-18  Almacén A  
2      2023